# Лабораторная работа №4: Классификация. Деревья решений

## Описание датасета

Датасет "Breast_Cancer.csv" содержит данные о пациентах с диагнозом рака молочной железы. Целевая переменная — `Status` (Alive/Dead), указывающая, выжил ли пациент. Признаки включают демографические данные, характеристики опухоли и результаты лечения:

- **Age**: Возраст пациента (число).
- **Race**: Раса (White, Black, Other).
- **Marital Status**: Семейное положение (Married, Single, Divorced, Widowed, Separated).
- **T Stage**: Стадия опухоли по TNM (T1, T2, T3, T4).
- **N Stage**: Стадия лимфатических узлов (N1, N2, N3).
- **6th Stage**: Общая стадия заболевания (IIA, IIB, IIIA, IIIB, IIIC).
- **differentiate**: Степень дифференцировки опухоли (Poorly differentiated, Moderately differentiated, Well differentiated, Undifferentiated).
- **Grade**: Грейд опухоли (1, 2, 3, 4).
- **A Stage**: Стадия по AJCC (Regional, Distant).
- **Tumor Size**: Размер опухоли в мм (число).
- **Estrogen Status**: Статус эстрогеновых рецепторов (Positive, Negative).
- **Progesterone Status**: Статус прогестероновых рецепторов (Positive, Negative).
- **Regional Node Examined**: Количество исследованных лимфоузлов (число).
- **Reginol Node Positive**: Количество поражённых лимфоузлов (число).
- **Survival Months**: Месяцы выживания (число).
- **Status**: Целевая переменная (Alive, Dead).

Задача: предсказать статус пациента (Alive/Dead) на основе признаков.

In [178]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Считывание и обзор данных


In [179]:
df = pd.read_csv('Breast_Cancer.csv')
print('Размерность:', df.shape)
print('Колонки:', df.columns.tolist())
display(df.head())
df.info()
print('Пропуски по столбцам:')
print(df.isnull().sum())

Размерность: (4024, 16)
Колонки: ['Age', 'Race', 'Marital Status', 'T Stage ', 'N Stage', '6th Stage', 'differentiate', 'Grade', 'A Stage', 'Tumor Size', 'Estrogen Status', 'Progesterone Status', 'Regional Node Examined', 'Reginol Node Positive', 'Survival Months', 'Status']


,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Reginol Node Positive   4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

## 2. Предобработка данных


In [180]:
# Удаляем пропуски
df = df.dropna()

# Кодируем целевую переменную
le = LabelEncoder()
y = le.fit_transform(df['Status'])  # Alive=0, Dead=1

# Отбрасываем неинформативные признаки
X = df.drop(columns=['Status', 'Survival Months'])

# Кодируем категориальные признаки
X = pd.get_dummies(X, drop_first=True)

# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разбиение выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

## 3. Дерево решений

GridSearchCV по DecisionTreeClassifier

In [181]:
param_grid_dt = {'max_depth': [None, 3, 5, 7, 10], 'max_features': [None, 'sqrt', 'log2']}
gs_dt = GridSearchCV(DecisionTreeClassifier(class_weight='balanced', random_state=42), param_grid_dt, cv=10, scoring='f1', n_jobs=-1)
gs_dt.fit(X_train, y_train)
best_dt = gs_dt.best_estimator_
print('Лучшие параметры DT:', gs_dt.best_params_)

Лучшие параметры DT: {'max_depth': 3, 'max_features': 'sqrt'}


Метрики на тренировке:


In [182]:
y_pred_tr = best_dt.predict(X_train)
print({
    'accuracy': accuracy_score(y_train, y_pred_tr),
    'precision': precision_score(y_train, y_pred_tr),
    'recall': recall_score(y_train, y_pred_tr),
    'f1': f1_score(y_train, y_pred_tr)
})

{'accuracy': 0.6178937558247903, 'precision': 0.243205574912892, 'recall': 0.7079107505070994, 'f1': 0.3620331950207469}


Важности признаков и отбор:

In [183]:
importances_dt = pd.Series(best_dt.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Top-10 важностей DT:\n', importances_dt.head(10))
threshold_dt = importances_dt.mean()
selected_dt = importances_dt[importances_dt > threshold_dt].index.tolist()
print('Отобранные признаки DT:', selected_dt)

Top-10 важностей DT:
 Reginol Node Positive                  0.279641
differentiate_Poorly differentiated    0.246989
N Stage_N3                             0.184334
Grade_2                                0.131906
Estrogen Status_Positive               0.080625
differentiate_Well differentiated      0.076505
6th Stage_IIB                          0.000000
A Stage_Regional                       0.000000
Grade_3                                0.000000
Grade_1                                0.000000
dtype: float64
Отобранные признаки DT: ['Reginol Node Positive', 'differentiate_Poorly differentiated', 'N Stage_N3', 'Grade_2', 'Estrogen Status_Positive', 'differentiate_Well differentiated']


Повторный GridSearchCV на X_train[:, selected]

In [184]:
X_tr_dt = X_train[:, [list(X.columns).index(f) for f in selected_dt]]
gs_dt2 = GridSearchCV(DecisionTreeClassifier(class_weight='balanced', random_state=42), param_grid_dt, cv=10, scoring='f1', n_jobs=-1)
gs_dt2.fit(X_tr_dt, y_train)
print('DT после фильтрации, параметры:', gs_dt2.best_params_)
y_pred_dt2 = gs_dt2.best_estimator_.predict(X_tr_dt)
print({
    'accuracy': accuracy_score(y_train, y_pred_dt2),
    'precision': precision_score(y_train, y_pred_dt2),
    'recall': recall_score(y_train, y_pred_dt2),
    'f1': f1_score(y_train, y_pred_dt2)
})

DT после фильтрации, параметры: {'max_depth': 3, 'max_features': 'sqrt'}
{'accuracy': 0.6722584653619137, 'precision': 0.2606473594548552, 'recall': 0.6206896551724138, 'f1': 0.367126574685063}


## 4. Случайный лес



In [185]:
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10], 'max_features': ['sqrt', 'log2']}
gs_rf = GridSearchCV(RandomForestClassifier(class_weight='balanced_subsample', random_state=42), param_grid_rf, cv=10, scoring='f1', n_jobs=-1)
gs_rf.fit(X_train, y_train)
best_rf = gs_rf.best_estimator_
print('Лучшие параметры RF:', gs_rf.best_params_)

# 4.2 Метрики на тренировке
y_pred_rf = best_rf.predict(X_train)
print({
    'accuracy': accuracy_score(y_train, y_pred_rf),
    'precision': precision_score(y_train, y_pred_rf),
    'recall': recall_score(y_train, y_pred_rf),
    'f1': f1_score(y_train, y_pred_rf)
})

# 4.3 Важности и фильтрация
importances_rf = pd.Series(best_rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Top-10 важностей RF:\n', importances_rf.head(10))
threshold_rf = importances_rf.mean()
selected_rf = importances_rf[importances_rf > threshold_rf].index.tolist()
print('Отобранные признаки RF:', selected_rf)

# 4.4 Повторный GridSearch
X_tr_rf = X_train[:, [list(X.columns).index(f) for f in selected_rf]]
gs_rf2 = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=10, scoring='recall', n_jobs=-1)
gs_rf2.fit(X_tr_rf, y_train)
print('RF после фильтрации, параметры:', gs_rf2.best_params_)
y_pred_rf2 = gs_rf2.best_estimator_.predict(X_tr_rf)
print({
    'accuracy': accuracy_score(y_train, y_pred_rf2),
    'precision': precision_score(y_train, y_pred_rf2),
    'recall': recall_score(y_train, y_pred_rf2),
    'f1': f1_score(y_train, y_pred_rf2)
})

Лучшие параметры RF: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 100}
{'accuracy': 0.7769493631562597, 'precision': 0.3619631901840491, 'recall': 0.5983772819472617, 'f1': 0.4510703363914373}
Top-10 важностей RF:
 Reginol Node Positive                  0.174719
6th Stage_IIIC                         0.102375
Progesterone Status_Positive           0.099299
N Stage_N3                             0.089799
Tumor Size                             0.084341
Age                                    0.076931
Estrogen Status_Positive               0.072470
differentiate_Poorly differentiated    0.046142
Regional Node Examined                 0.044115
Grade_3                                0.037282
dtype: float64
Отобранные признаки RF: ['Reginol Node Positive', '6th Stage_IIIC', 'Progesterone Status_Positive', 'N Stage_N3', 'Tumor Size', 'Age', 'Estrogen Status_Positive', 'differentiate_Poorly differentiated', 'Regional Node Examined', 'Grade_3']
RF после фильтрации, параметры: {'max_d

## 5. Метод ближайших соседей


In [186]:
param_grid_knn = {'n_neighbors': list(range(3, 16, 2))}
gs_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=10, scoring='f1', n_jobs=-1)
gs_knn.fit(X_train, y_train)
best_knn = gs_knn.best_estimator_
print('Лучшие параметры KNN:', gs_knn.best_params_)

# 5.2 Метрики на тренировке
y_pred_knn = best_knn.predict(X_train)
print({
    'accuracy': accuracy_score(y_train, y_pred_knn),
    'precision': precision_score(y_train, y_pred_knn),
    'recall': recall_score(y_train, y_pred_knn),
    'f1': f1_score(y_train, y_pred_knn)
})

Лучшие параметры KNN: {'n_neighbors': 3}
{'accuracy': 0.893134513824169, 'precision': 0.7811320754716982, 'recall': 0.4198782961460446, 'f1': 0.5461741424802111}


## 6. Пересечение признаков

In [187]:
improved = (
    gs_dt2.best_score_ > gs_dt.best_score_ or
    gs_rf2.best_score_ > gs_rf.best_score_
    )
common_feats = list(set(selected_dt) & set(selected_rf) & set(importances_rf.head(10).index)) if improved else []
print('Общие признаки для финальной модели:', common_feats)

Общие признаки для финальной модели: ['Estrogen Status_Positive', 'Reginol Node Positive', 'N Stage_N3', 'differentiate_Poorly differentiated']


## 7. Оценка на тестовой выборке и визуализация

In [188]:
# 7.1 Оценка моделей на тесте
models = {'DT': best_dt, 'RF': best_rf, 'KNN': best_knn}
for name, m in models.items():
    y_pred = m.predict(X_test)
    print(f"{name} Test:", {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred)
    })

# 7.2 Текстовая визуализация дерева (depth=3)
small_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
small_tree.fit(X_train, y_train)
print(export_text(small_tree, feature_names=list(X.columns)))

DT Test: {'accuracy': 0.5937888198757764, 'precision': 0.211864406779661, 'recall': 0.6097560975609756, 'f1': 0.31446540880503143}
RF Test: {'accuracy': 0.737888198757764, 'precision': 0.2962962962962963, 'recall': 0.5203252032520326, 'f1': 0.3775811209439528}
KNN Test: {'accuracy': 0.8273291925465839, 'precision': 0.375, 'recall': 0.1951219512195122, 'f1': 0.25668449197860965}
|--- N Stage_N3 <= 1.19
|   |--- Estrogen Status_Positive <= -1.73
|   |   |--- Tumor Size <= 0.10
|   |   |   |--- class: 0
|   |   |--- Tumor Size >  0.10
|   |   |   |--- class: 1
|   |--- Estrogen Status_Positive >  -1.73
|   |   |--- Reginol Node Positive <= 0.07
|   |   |   |--- class: 0
|   |   |--- Reginol Node Positive >  0.07
|   |   |   |--- class: 0
|--- N Stage_N3 >  1.19
|   |--- Estrogen Status_Positive <= -1.73
|   |   |--- Regional Node Examined <= -0.11
|   |   |   |--- class: 0
|   |   |--- Regional Node Examined >  -0.11
|   |   |   |--- class: 1
|   |--- Estrogen Status_Positive >  -1.73
|  

In [189]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

ensemble = VotingClassifier(estimators=[
    ('dt', DecisionTreeClassifier(class_weight='balanced')),
    ('rf', RandomForestClassifier(class_weight='balanced_subsample')),
    ('knn', KNeighborsClassifier(weights='distance', n_neighbors=3))
], voting='soft')

ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)


In [190]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(class_weight='balanced_subsample', random_state=42))
])

param_grid = {
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [5, 7],
    'rf__min_samples_leaf': [10, 20]
}

gs = GridSearchCV(pipe, param_grid, scoring='precision', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

# Сдвигаем порог вероятностей:
y_pred_proba = gs.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.25).astype(int)

print('Итоговые метрики после оптимизации порога:')
print({
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1': f1_score(y_test, y_pred)
})


Итоговые метрики после оптимизации порога:
{'accuracy': 0.21739130434782608, 'precision': 0.16245006657789615, 'recall': 0.991869918699187, 'f1': 0.2791762013729977}


## Выводы

- **DecisionTreeClassifier**: Показал [указать результаты, например, высокую точность, но склонность к переобучению].
- **RandomForestClassifier**: Дал [указать результаты, например, лучшие метрики за счёт ансамблевого подхода].
- **KNeighborsClassifier**: [Указать результаты, например, чувствительность к выбору k].
- Фильтрация признаков [улучшила/не улучшила] метрики для [указать моделей].
- На тестовой выборке лучшей моделью оказалась [указать модель] с метриками [указать значения].